### Imports

In [11]:
import numpy as np
import pandas as pd
import importlib
import os

import findatree.io as io
import findatree.transformations as transformations
import findatree.interactive as interactive
import findatree.geo_to_image as geo_to_image
import findatree.segmentation as segmentation
import findatree.photometric.shadow as shadow
import findatree.photometric.features as features

from bokeh.plotting import show, save
from bokeh.io import output_notebook, output_file

output_notebook()

Loading BokehJS ...

### Definitions

In [19]:
tnr = 9996

dir_names = []
dir_names.extend(['/home/flostehr/Data/lwf/WZE-UAV-2020/DSMs'])
dir_names.extend(['/home/flostehr/Data/lwf/WZE-UAV-2020/DTMs'])
dir_names.extend(['/home/flostehr/Data/lwf/WZE-UAV-2020/Orthos'])
dir_names.extend(['/home/flostehr/Data/lwf/WZE-UAV-2020/Kronen'])


dir_name_hdf5 = "/home/flostehr/Data/lwf/processed/2020/hdf5"
dir_name_html = "/home/flostehr/Data/lwf/analysis/examples"

### Load channels from .tif

In [21]:
importlib.reload(geo_to_image)

params_channels = {
    'tnr': tnr,
    'px_width_reproject': 0.2,
    'downscale': 0,
}

# Load, reproject, normalize & downscale rasters
channels, params_channels = geo_to_image.channels_load(
    dir_names,
    params_channels,
)

# Save repojected rasters as channels in .hdf5
# io.channels_to_hdf5(channels, params_channels, dir_name = dir_name_hdf5)

### Load channels from .hdf5

In [22]:
importlib.reload(io)

# Load data
data, params_data = io.load_hdf5(
    path=os.path.join(dir_name_hdf5, f"tnr{tnr}.hdf5"),
    groups=['channels'],
)

# Assign channels
channels = data['channels']
params_channels = params_data['channels']
del data, params_data

### Load human crowns

In [23]:
importlib.reload(io)

# Load 'hand selected' human crowns
crowns_human, params_crowns_human = io.load_shapefile(
    dir_names,
    params_channels,
    remove_outliers = True,
    )

# Save crowns in .hdf5
io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_name_hdf5)

-----------
Parameters:
  affine                        : [[ 2.00000000e-01  0.00000000e+00  7.51356393e+05]
 [ 0.00000000e+00 -2.00000000e-01  5.43435026e+06]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
  date_time_polygons            : 230225-231933
  date_time_terrestrial         : 230225-231933
  number_crowns                 : 40
  origin                        : human
  path_shapes                   : /home/flostehr/Data/lwf/WZE-UAV-2020/Kronen/Tnr_9996_Kr.shp
  shape                         : [1750 1611]
  tnr                           : 9996


### Add photometric features to human crowns

In [24]:
importlib.reload(features)
importlib.reload(io)

# Add photometric features to human crowns
features.crowns_add_features(
    channels,
    params_channels,
    crowns_human,
    params_crowns_human,
)
# Save crowns in .hdf5
io.crowns_to_hdf5(crowns_human, params_crowns_human, dir_name = dir_name_hdf5)

### View human crown features as DataFrame

In [26]:
df = pd.merge(
    left = pd.DataFrame(crowns_human['features']['photometric']),
    right = pd.DataFrame(crowns_human['features']['terrestrial']),
    how= 'outer',
    on = ['id'],
    )

# df.loc[:,['shape_area', 'area']]

,shape_area,area
0,78.190506,78.080002
1,58.440033,58.359997
2,50.222984,50.119999
3,46.009453,47.399998
4,47.758369,47.320000
5,22.165644,21.760000
6,28.039381,29.240000
7,32.335281,31.799999
8,23.821590,23.559999
9,26.475212,27.480000


### Plotting

In [ ]:
importlib.reload(interactive)

plt = interactive.Plotter()
plt.width = 400
plt.channels_downscale = 0
plt.show_features = [
    'id', 'ba', 'kkl','nbv',
    'perc5_ndre','perc50_ndre'
]

plt.add_channels(channels, params_channels)

plt.figures_add_rgb(perc=2)
plt.figures_add_rgb(perc=2, color_code='533', weight_code = '133')
# plt.figures_add_gray('mask')

plt.togglers_add_crowns(crowns_human, params_crowns_human)

layout = plt.create_layout()

In [ ]:
show(layout)

# output_file(os.path.join(dir_name_html, f"tnr{tnr}.html"), mode='inline')
# save(layout)